# Day 17: Pyroclastic Flow

[*Advent of Code 2022 day 17*](https://adventofcode.com/2022/day/17) and [*solution megathread*](https://redd.it/znykq2)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/17/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F17%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [4]:
testdata = """>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"""

inputdata = downloaded['input']
# inputdata = open('input.txt', 'r').read().splitlines()

In [5]:
from IPython.display import display

display(f'{len(testdata)=}')

display(f'{inputdata[:10]} ... {len(inputdata)=}')

'len(testdata)=40'

'><<><>><<< ... len(inputdata)=10091'

In [11]:
from typing import Tuple

Shape = Tuple[int, ...]


def tetrominos() -> Tuple[Shape, ...]:
    def str_to_int(row: str) -> int:
        return eval('0b00' + row + '0'*(5 - len(row)))

    basic_shapes = (('1111'),
                    ('010',
                     '111',
                     '010'),
                    ('001',
                     '001',
                     '111'),
                    ('1',
                     '1',
                     '1',
                     '1'),
                    ('11',
                     '11'))

    return tuple(tuple(str_to_int(row) for row in shape[::-1])
                 for shape in basic_shapes)

In [12]:
[[f'{row:07b}' for row in shape] for shape in tetrominos()]

[['0010000', '0010000', '0010000', '0010000'],
 ['0001000', '0011100', '0001000'],
 ['0011100', '0000100', '0000100'],
 ['0010000', '0010000', '0010000', '0010000'],
 ['0011000', '0011000']]

In [8]:
from typing import TypeVar, Collection, Iterable
from itertools import cycle
# from collections.abc import Iterable, Sized

G = TypeVar('G')


def generator(data: Collection[G]) -> Iterable[Tuple[int, G]]:
    return cycle(enumerate(data, start=1))

In [9]:
jets = iter(generator(testdata))
for _ in range(5):
    display(next(jets))

(1, '>')

(2, '>')

(3, '>')

(4, '<')

(5, '<')

In [27]:
from typing import List, Iterator


def drop_shape(occupied: List[int],
               shape: Shape,
               jets: Iterator[Tuple[int, str]]) \
            -> Tuple[int, Shape, int]:
    def is_clear(test_row: int, test_shape: Shape) -> bool:
        for row_in_shape in range(len(test_shape)):
            if test_row + row_in_shape >= len(occupied):
                return True
            if occupied[test_row + row_in_shape] & test_shape[row_in_shape]:
                return False
        return True

    def apply_jet() -> Shape:
        if jet == '>' and not any(
                shape_line & 0b0000001 for shape_line in shape):
            test_shape = tuple(shape_line >> 1 for shape_line in shape)
        elif jet == '<' and not any(
                shape_line & 0b1000000 for shape_line in shape):
            test_shape = tuple(shape_line << 1 for shape_line in shape)
        if is_clear(row, test_shape):
            return test_shape
        return shape

    row = len(occupied) + 3
    while row > 0:
        i_jet, jet = next(jets)
        shape = apply_jet()
        if not is_clear(row - 1, shape):
            break
        row -= 1
    return row, shape, i_jet

In [ ]:
def place_shape(occupied: List[int], shape: Shape):
    def generate_tile(x: int, y: int, current_tile: str = '.') -> str:
        if current_tile != '.':
            return current_tile
        elif (x, y) in rendered_shape:
            return '#'
        else:
            return '.'
    if occupied:
        width = len(occupied[0])
    else:
        width = 7
    bottom = min(c[1] for c in rendered_shape)
    top = max(c[1] for c in rendered_shape)
    for y in range(bottom, top + 1):
        if y == len(occupied):
            occupied.append(''.join(generate_tile(x, y)
                                    for x in range(width)))
        else:
            occupied[y] = ''.join(generate_tile(x, y, occupied[y][x])
                                  for x in range(width))

In [ ]:

jets = iter(generate_jets(testdata))
shapes = iter(generate_tetrominos())
occupied: List[str] = list()
for i in count(start=1):
# for i in range(1, 2022 + 1):
    i_jet, next_shape = drop_shape(occupied, next(shapes), jets)
    place_shape(occupied, next_shape)
    if i % 5 == 0 and i_jet == len(testdata) - 1:
        display(f'{i}: {len(occupied)}')
        break
len(occupied)

KeyboardInterrupt: 

7:1: E115 expected an indented block (comment)


In [ ]:
len(occupied)

703533

In [ ]:
jets = iter(generate_jets(inputdata))
shapes = iter(generate_tetrominos())
occupied: List[str] = list()
for i in range(2022):
    place_shape(occupied, drop_shape(occupied, next(shapes), jets))
len(occupied)

3173

['....#..',
 '....#..',
 '....##.',
 '##..##.',
 '######.',
 '.###...',
 '..#....',
 '.####..',
 '....##.',
 '....##.',
 '....#..',
 '..#.#..',
 '..#.#..',
 '#####..',
 '..###..',
 '...#...',
 '...####']

In [ ]:

Line = Tuple[Coord, ...]


def parse_lines(data: Iterable[str]) -> List[Line]:
    output: List[Line] = list()
    for line in data:
        line_coords: List[Coord] = list()
        for coord in line.split(' -> '):
            x, y = coord.split(',', 1)
            line_coords.append((int(x), int(y)))
        output.append(tuple(line_coords))
    return output

In [ ]:
from typing import Set, Iterable


def generate_lines(lines: Iterable[Line]) -> Set[Coord]:
    def generate_pixels(dim_start: int, dim_end) -> Iterable[int]:
        step = 1
        if dim_start >= dim_end:
            step = -step
        return range(dim_start, dim_end + step, step)

    def generate_segment(c_start: Coord, c_end: Coord) -> Iterable[Coord]:
        if c_start[0] == c_end[0]:
            return ((c_start[0], c1)
                    for c1 in generate_pixels(c_start[1], c_end[1]))
        elif c_start[1] == c_end[1]:
            return ((c0, c_start[1])
                    for c0 in generate_pixels(c_start[0], c_end[0]))
        else:
            raise ValueError

    def generate_line(coords: Line) -> Iterable[Coord]:
        return (pixel
                for c_start, c_end in zip(coords[:-1], coords[1:])
                for pixel in generate_segment(c_start, c_end))

    return {pixel
            for line in lines
            for pixel in generate_line(line)}

In [ ]:
def scene_str(stone: Set[Coord], occupied: Set[Coord] = set()) -> List[str]:
    def window(coords: Set[Coord]) -> Tuple[Coord, Coord]:
        min_x = min(c[0] for c in coords)
        max_x = max(c[0] for c in coords)
        min_y = min(c[1] for c in coords)
        max_y = max(c[1] for c in coords)
        return ((min_x - 1, min_y - 1), (max_x + 1, max_y + 1))

    def pixel_chr(pixel: Coord) -> str:
        if pixel in stone:
            return '#'
        elif pixel in occupied:
            return 'o'
        else:
            return '.'

    # replace with occupied.union(stone)?
    w = window(stone.union(occupied))
    return [
        ''.join(pixel_chr((x, y)) for x in range(w[0][0], w[1][0] + 1))
        for y in range(w[0][1], w[1][1] + 1)]

In [ ]:
from typing import Optional


def drop_grain(
        stone: Set[Coord],
        occupied: Set[Coord] = set(),
        pos: Coord = (500, 0),
        abyss: int = -1,
        floor_not_abyss: bool = False) -> Optional[Coord]:
    if abyss == -1:
        abyss = max(c[1] + 2 for c in stone)
    # Let's see, if either of the next positions are free,
    # in that order of preference, the grain will fall
    # there, and only if not, it will come to rest where it
    # is
    if floor_not_abyss and pos[1] + 1 >= abyss:
        return pos
    if pos[1] >= abyss or pos in occupied:
        return None
    next_positions = [
        (pos[0], pos[1] + 1),
        (pos[0] - 1, pos[1] + 1),
        (pos[0] + 1, pos[1] + 1)]
    for next_pos in next_positions:
        if next_pos not in occupied:
            return drop_grain(stone,
                              occupied,
                              next_pos,
                              abyss,
                              floor_not_abyss)
    return pos

In [ ]:
stone = generate_lines(parse_lines(testdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))
display(scene_str(stone, occupied))

24

['............',
 '.......o....',
 '......ooo...',
 '.....#ooo##.',
 '....o#ooo#..',
 '...###ooo#..',
 '.....oooo#..',
 '..o.ooooo#..',
 '.#########..',
 '............']

In [ ]:
stone = generate_lines(parse_lines(inputdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))

755

In [ ]:
HTML(downloaded['part1_footer'])

## Part Two

In [ ]:
HTML(downloaded['part2'])

In [ ]:
stone = generate_lines(parse_lines(testdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied, floor_not_abyss=True):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))
display(scene_str(stone, occupied))

93

['.......................',
 '...........o...........',
 '..........ooo..........',
 '.........ooooo.........',
 '........ooooooo........',
 '.......oo#ooo##o.......',
 '......ooo#ooo#ooo......',
 '.....oo###ooo#oooo.....',
 '....oooo.oooo#ooooo....',
 '...oooooooooo#oooooo...',
 '..ooo#########ooooooo..',
 '.ooooo.......ooooooooo.',
 '.......................']

In [ ]:
stone = generate_lines(parse_lines(inputdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied, floor_not_abyss=True):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))

29805

In [ ]:
HTML(downloaded['part2_footer'])